<a href="https://colab.research.google.com/github/ort-eila/csiseminar/blob/main/project/step2_DataToVec_mibi_DCIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount("/content/gdrive")
# csiseminar/project/step2_DataToVec_mibi_DCIS.ipynb

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
import os
root_dir = "/content/gdrive/MyDrive/Seminar/project/immune_310_project"
dataset_path=os.path.join(root_dir,"dataset","NPY_41_channels","labels")
dataset_path

'/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels'

In [ ]:
progressors_npy_dir = os.path.join(dataset_path,"progressor")
print("progressors_npy_dir is {}".format(progressors_npy_dir))
non_progressors_npy_dir = os.path.join(dataset_path,"nonprogressor")
print("non_progressors_npy_dir is {}".format(non_progressors_npy_dir))


In [ ]:
progressors_npy_files = os.listdir(progressors_npy_dir)
non_progressors_npy_files = os.listdir(non_progressors_npy_dir)
npy_n_channels_all = [os.path.join(progressors_npy_dir,x) for x in progressors_npy_files]
npy_n_channels_all = npy_n_channels_all + [os.path.join(non_progressors_npy_dir,x) for x in non_progressors_npy_files]
print(npy_n_channels_all[0:5])
print(npy_n_channels_all[-5:])


In [ ]:
# Step 2. create 224x224 images for the model and save them in a seperate foldet
# with a folder name = dataset name

In [ ]:
import numpy as np
from PIL import Image
import re
import os

def create_rgb_image_from_channels(channel_data, output_shape, output_path, filename_prefix):
    height, width = channel_data[0].shape

    grayscale_vector = []
    #   print("channel_index is {}".format(channel_index))
    for col in range(0, width):
      # print("!!!!col is {}. len(grayscale_vector) is {}".format(col,len(grayscale_vector)))
      for row in range(0, height):
        # print("row is {}".format(row))
        for channel_index in range(0, len(channel_data)-1):
          # print("channel_index is {}".format(channel_index))
          val = channel_data[channel_index][row, col]
          # print("channel_index is {}".format(channel_index))
          # print("row is {}".format(row))
          # print("col is {}".format(col))
          # print("val is {}".format(val))
          # print("adding val value {}".format(val))
          grayscale_vector.append(val)
      # Calculate the sum of the vector elements
      # print("number of elements in vector is {}".format(len(grayscale_vector)))
      vector_sum = sum(grayscale_vector)
      if vector_sum == 0:
        # print("The vector is all zeros.")
        continue
      # else:
      #   print("The vector is not all zeros.")

      # print("matrix")
      # Calculate the required number of elements for the desired image size
      required_elements = output_shape[0]*output_shape[1] * 3
      # print("required_elements are {}".format(required_elements))
      rgb_image_matrix = None
      # Check if vector needs padding
      if len(grayscale_vector) < required_elements:
          # Calculate the number of elements to pad
          padding_elements = required_elements - len(grayscale_vector)
          # print("len(padding_elements) is {}".format(padding_elements))
          # Pad the vector with zeros
          padded_vector = np.concatenate((grayscale_vector, np.zeros(padding_elements)))
          # print("len(padded_vector) is {}".format(len(padded_vector)))
          # Reshape the padded vector into a 224x224 array
          rgb_image_matrix = padded_vector.reshape(output_shape[0], output_shape[1],3)
          # print("len(rgb_image_matrix) is {}".format(len(rgb_image_matrix)))
      else:
          # Reshape the vector into a 224x224 array
          print("else. grayscale_vector size is {}. required_elements is {}".format(len(grayscale_vector),required_elements))
          rgb_image_matrix = np.array(grayscale_vector[0:required_elements]).reshape(output_shape[0], output_shape[1],3)

      # Convert to PIL Image
      # print("before pil_image")
      # Convert to PIL Image
      pil_image = Image.fromarray(rgb_image_matrix.astype(np.uint8))

      # Save the image as a JPEG file
      # print("before file_name")
      file_name = f'{filename_prefix}_vector_image_row_{row}_col{col}.jpg'
      # output_path = 'output_image.jpg'  # Change this to your desired output path with .jpg extension
      full_file_name = f'{output_path}/{file_name}'
      # print("before save")
      pil_image.save(full_file_name) #, format='JPEG')
      grayscale_vector = []


In [ ]:
# npy_n_channels_all
# required_elements - len(grayscale_vector)

In [ ]:

# # execution example
# # Combine channel data
# channel_data = [channel1, channel2, channel3, channel4, channel5, channel6, channel7, channel8, channel9, channel10]
# # Define output path and filename prefix
# output_path = os.path.join(root_dir,'output_folder')
# print("output_path is {}".format(output_path))

# os.makedirs(output_path,exist_ok=True)
# filename_prefix = 'output_prefix'
# print("filename_prefix is {}".format(filename_prefix))
# output_shape = (224, 224)

import re
for npz_file_path in npy_n_channels_all:
    # Replace the pattern using regular expressions
    print("npz_file_path is {}".format(npz_file_path))
    output_shape = (224, 224)
    # num_channels = 42
    npz_file_fld = os.path.dirname(npz_file_path)
    print("npz_file_fld is {}".format(npz_file_fld))
    # output_path = npz_file_fld.replace("NPY_41_channels","JPG_RGB_224_224")
    output_path = re.sub(r'NPY_.*_channels', 'JPG_VECT_TO_IMG_{}_{}'.format(output_shape[0], output_shape[1]), npz_file_fld)
    print("output_path is {}".format(output_path))
    os.makedirs(output_path, exist_ok=True)
    patient = os.path.basename(npz_file_path).split(".npz")[0]
    # Load the NPZ file
    # Load the NPZ file
    import numpy as np

    # Load the NPZ file
    loaded_data = np.load(npz_file_path)

    # Check the keys present in the loaded NPZ file
    print("Keys in the NPZ file:", loaded_data.files)

    # Now you have the extracted channel_data as a list of numpy arrays
    channel_data = loaded_data['channel_data']
    num_channels = len(channel_data)
    # Call the function to create and save RGB images
    create_rgb_image_from_channels(channel_data, output_shape, output_path, patient)


In [ ]:
# len(channel_data)

In [ ]:
# output_shape = (224, 224)
# output_shape[0]*output_shape[1]

In [ ]:
import numpy as np

# Generate random data
np.random.seed(42)  # For reproducibility
height, width, num_channels = 1024, 1024, 3
random_data = np.random.randint(0, 256, size=(height, width, num_channels), dtype=np.uint8)

# Save the random data to an NPZ file
np.savez_compressed('random_data.npz', array=random_data)

# Load the NPZ file
data = np.load('random_data.npz')
array = data['array']

# Reshape the array to a 2D shape (number of pixels, number of channels)
reshaped_array = array.reshape(height * width, num_channels)

# Now you have a 2D array where each row corresponds to a pixel's vector
# For example, let's print the pixel vector of the first pixel
first_pixel_vector = reshaped_array[0]
print("Pixel Vector for the First Pixel:", first_pixel_vector)

# You can also iterate over all pixel vectors
for pixel_vector in reshaped_array:
    # Do something with the pixel_vector
    pass


In [ ]:
# output_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_VECT_TO_IMG_224_224/labels/progressor"
# output_path

In [ ]:
# os.path.dirname(output_path)

In [7]:
# create the train / validation and test folder for the created images:

import os
import shutil
import random

# Define the root folder containing the labels and images
root_folder = os.path.dirname(output_path)
print("root_folder is {}".format(root_folder))
output_folder = os.path.join(os.path.dirname(root_folder),"train_validation_test") # Define the output folder where train, validation, and test sets will be created
print("output_folder is {}".format(output_folder))

# Define label names
label_names = os.listdir(root_folder)
print("label_names is {}".format(label_names))
# Define train, validation, and test ratios
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Create train, validation, and test folders
for split in ['train', 'validation', 'test']:
  print("1")
  split_folder = os.path.join(output_folder, split)
  os.makedirs(split_folder, exist_ok=False)
  print("2")
  for label in label_names:
      print("3")
      os.makedirs(os.path.join(split_folder, label), exist_ok=False)

# Iterate through each label
# will start from all the progressors for all datasets and then will continue to the non progressors
for label in label_names:
  print("4")
  label_folder = os.path.join(root_folder, label)
  image_files = [f for f in os.listdir(label_folder) if f.endswith('.jpg')]

  random.shuffle(image_files)

  num_samples = len(image_files)
  num_train = int(num_samples * train_ratio)
  num_validation = int(num_samples * validation_ratio)

  train_images = image_files[:num_train]
  validation_images = image_files[num_train:num_train + num_validation]
  test_images = image_files[num_train + num_validation:]

  for image in train_images:
      src = os.path.join(label_folder, image)
      dest = os.path.join(output_folder, 'train', label, image)
      shutil.copy(src, dest)
      print("5")

  for image in validation_images:
      src = os.path.join(label_folder, image)
      dest = os.path.join(output_folder, 'validation', label, image)
      shutil.copy(src, dest)
      print("6")

  for image in test_images:
      src = os.path.join(label_folder, image)
      dest = os.path.join(output_folder, 'test', label, image)
      shutil.copy(src, dest)
      print("7")

print("Data splitting and copying complete.")



6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
